In [62]:
import numpy as np
import pandas as pd
import os

In [63]:
def loadCSV(filename):
    file = filename
    if '.csv' not in filename:
        file += '.csv'
    data = pd.read_csv(file, encoding = 'ISO-8859-1')
    return data

In [64]:
csvFile = "altered_hate_speech"
data = loadCSV(csvFile)

In [65]:
data.head()

,tweet_text,confidence,tweet_class
0,Warning: penny boards will make you a faggot,0.6013,1
1,Fuck dykes,0.7227,2
2,@sizzurp__ @ILIKECATS74 @yoPapi_chulo @brandon...,0.5229,2
3,"""@jayswaggkillah: ""@JacklynAnnn: @jayswaggkill...",0.5184,2
4,@Zhugstubble You heard me bitch but any way I'...,0.5185,1


In [66]:
data.tail()

,tweet_text,confidence,tweet_class
14504,I'm sorry. Did I offend your white supremacist...,0.3418,0
14505,@tradethecycles Caucasian euro aryan whatever....,0.6804,0
14506,@yadavakhilesh sir a patient named aryan khan ...,1.0000,0
14507,@Iamshivachari Happy birthday bro _ÃÂ«ÃÃ_ ...,1.0000,0
14508,@Hajrahkiayni Aryan Kapoor is such a cute name...,1.0000,0


# Transform Tweets to Lowercase

Make all characters in tweet_text lowercase.

In [67]:
data["tweet_text"] = data["tweet_text"].apply(lambda x: x.lower())

In [68]:
data.head()

,tweet_text,confidence,tweet_class
0,warning: penny boards will make you a faggot,0.6013,1
1,fuck dykes,0.7227,2
2,@sizzurp__ @ilikecats74 @yopapi_chulo @brandon...,0.5229,2
3,"""@jayswaggkillah: ""@jacklynannn: @jayswaggkill...",0.5184,2
4,@zhugstubble you heard me bitch but any way i'...,0.5185,1


In [69]:
data.tail()

,tweet_text,confidence,tweet_class
14504,i'm sorry. did i offend your white supremacist...,0.3418,0
14505,@tradethecycles caucasian euro aryan whatever....,0.6804,0
14506,@yadavakhilesh sir a patient named aryan khan ...,1.0000,0
14507,@iamshivachari happy birthday bro _ãâ«ãã_ ...,1.0000,0
14508,@hajrahkiayni aryan kapoor is such a cute name...,1.0000,0


# Filter, Distill, and Standardize Tweets

1) Extra whitespace removed

2) Punctuation removed

3) Hashtag character removed

3) Garbage Characters removed

4) Numbers removed

5) @[twitter_handle] replaced with [@]

6) http[link] replaced with [LINK]

In [70]:
validCharacters = 'abcdefghijklmnopqrstuvwxyz@'

In [71]:
for row in range(0, len(data["tweet_text"])):
    original_tweet = data["tweet_text"][row]
    words = original_tweet.split()
    
    for i in range(0, len(words)):
        if "@" in words[i]:
            words[i] = "[@]"
        elif "http" in words[i]:
            words[i] = "[LINK]"
        else:
            filtered_word = ''.join(filter(lambda x: x in validCharacters, words[i]))
            words[i] = filtered_word
    
    filtered_tweet = " ".join(words)
    
    data["tweet_text"] = data["tweet_text"].replace(original_tweet, filtered_tweet)

In [72]:
data.head()

,tweet_text,confidence,tweet_class
0,warning penny boards will make you a faggot,0.6013,1
1,fuck dykes,0.7227,2
2,[@] [@] [@] [@] [@] at least i dont look like ...,0.5229,2
3,[@] [@] [@] is a fag jackie jealous neeeee,0.5184,2
4,[@] you heard me bitch but any way im back th ...,0.5185,1


In [73]:
data.tail()

,tweet_text,confidence,tweet_class
14504,im sorry did i offend your white supremacist a...,0.3418,0
14505,[@] caucasian euro aryan whatever really doesn...,0.6804,0
14506,[@] sir a patient named aryan khan village mee...,1.0000,0
14507,[@] happy birthday bro have an happy year ahead,1.0000,0
14508,[@] aryan kapoor is such a cute name tho d we...,1.0000,0


# Calculate Frequency of Unique Words By Class

Determine frequency of unique words in all, neutral, offensive, and hate type tweets.
This is computed after preliminary preprocessing.

In [74]:
def iterateFreq(dictionary, word):
    if dictionary.get(word, False):
        dictionary[word] += 1
    else:
        dictionary[word] = 1

In [75]:
def wordFrequencies(dataframe):
    all_tweets = {}
    neutral = {}
    offensive = {}
    hate = {}
    
    for index, row in dataframe.iterrows():
        tweet = row[0]
        tweet_class = row[2]
        
        tweet_words = tweet.split()
        
        for word in tweet_words:
            if tweet_class == 0:
                iterateFreq(neutral, word)
            elif tweet_class == 1:
                iterateFreq(offensive, word)
            elif tweet_class == 2:
                iterateFreq(hate, word)
                
            iterateFreq(all_tweets, word)
    
    return all_tweets, neutral, offensive, hate

In [76]:
def dispWordFreq(dictionary):
    unique_words = sorted( ((v,k) for k, v in dictionary.items()), reverse = True)
    for v, k in unique_words:
        word_val = "%s: %d" % (k, v)
        print(word_val)

In [77]:
all_wf, neutral_wf, offensive_wf, hate_wf = wordFrequencies(data)

In [78]:
len(all_wf), len(neutral_wf), len(offensive_wf), len(hate_wf)

(17213, 12903, 7169, 4264)

# Save Altered CSV

In [79]:
path = os.getcwd()

In [80]:
new_csv = "/processed_hate_speech.csv"
filepath = path + new_csv

In [81]:
data.to_csv(filepath, index = None, header = True)